# Playground

## GNN
This code receives multiple timeseries, transforms them into graphs, and then applies a GNN to them.
The graph embeddings are then used for downstream tasks.

In [2]:
%reload_ext autoreload
%autoreload 3

import torch
from torch import nn
from torchinfo import summary
import pytorch_lightning as pl
from src.b2bnet import B2BNetModel, RandomDataModule, OtkaDataModule
from pytorch_lightning.loggers import TensorBoardLogger

In [6]:
# VISUALIZE model's graph

segment_size = 120*2
batch_size = 256

datamodule = OtkaDataModule(segment_size=segment_size, batch_size=batch_size)
model = B2BNetModel(
    input_size=59,
    n_timesteps=segment_size,
    n_cls_labels=2,
    hidden_size=8,
    n_subjects=51)

# summary(model)
# TRAINING
trainer = pl.Trainer(max_epochs=2, accelerator='cpu', log_every_n_steps=1, enable_checkpointing=True)
trainer.fit(model, datamodule=datamodule, ckpt_path='last')

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name              | Type             | Params
-------------------------------------------------------
0 | subject_embedding | Embedding        | 204   
1 | encoder_fc        | Linear           | 512   
2 | encoder           | RNN              | 34.6 K
3 | decoder           | RNN              | 34.6 K
4 | decoder_fc        | Linear           | 531   
5 | fc                | Linear           | 18    
6 | b2b               | RNN              | 34.6 K
7 | loss_cls          | CrossEntropyLoss | 0     
8 | loss_reconn       | MSELoss          | 0     
9 | loss_b2b          | MSELoss          | 0     
-------------------------------------------------------
104 K     Trainable params
0         Non-trainable params
104 K     Total params
0.420     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

## TCN

In [ ]:
%reload_ext autoreload
%autoreload 3

import torch
from src.b2bnet import TCN
from torchinfo import summary

In [ ]:
batch_size = 32
n_timesteps = 128 * 320
n_features = 59
X = torch.randn(batch_size, n_timesteps, n_features).permute(0, 2, 1)
model = TCN(n_timesteps, output_length=128, n_features=n_features, kernel_size=8, dilation_base=2)
model(X).shape